#  Unit 2.4a hacks
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- image: /images/python.png
- categories: []
- type: ap
- week: 26

In [170]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

In [171]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'ice'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _appName = db.Column(db.String(255), unique=False, nullable=False)
    _personalUse = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _favoriteFeature = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)


    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, favoriteFeature, appName, personalUse, uid, password="123qwerty"):
        self._favoriteFeature = favoriteFeature
        self._appName = appName    # variables with self prefix become part of the object, 
        self._personalUse = personalUse
        self._uid = uid
        self.set_password(password)

#_____________________________________________________________________________________________

# a name getter method, extracts name from object
    @property
    def appName(self):
        return self._appName
    
    # a setter function, allows name to be updated after initial object creation
    @appName.setter
    def name(self, appName):
        self._appName = appName
#_____________________________________________________________________________________________


# a name getter method, extracts name from object
    @property
    def personalUse(self):
        return self._personalUse
    
    # a setter function, allows name to be updated after initial object creation
    @personalUse.setter
    def name(self, personalUse):
        self._appName = personalUse
#_____________________________________________________________________________________________

    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid

    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    
    #_____________________________________________________________________________________________

    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional method used for setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter against stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
   #_____________________________________________________________________________________________

# a name getter method, extracts name from object
    @property
    def favoriteFeature(self):
        return self._favoriteFeature
    
    # a setter function, allows name to be updated after initial object creation
    @favoriteFeature.setter
    def name(self, favoriteFeature):
        self._favoriteFeature = favoriteFeature
#_____________________________________________________________________________________________


    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "appName": self.appName,
            "personalUse": self.personalUse,
            "password": self.password,
            "favoriteFeature": self.favoriteFeature,
            "uid": self.uid,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, appName = "", personalUse = "", favoriteFeature="",password="", uid=""):
        """only updates values with length"""
        if len(appName) > 0:
            self.appName = appName
        if len(personalUse) > 0:
            self.personalUse = personalUse
        if len(favoriteFeature) > 0:
            self.favoriteFeature = favoriteFeature
        if len(password) > 0:
            self.set_password(password)
        if len(uid) > 0:
            self.uid = uid        
        db.session.add(self) # performs update when id exists
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

In [172]:
# Creating my own database 

In [173]:


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(appName='Netflix', personalUse='Watch Movies',favoriteFeature = "watching Movies", uid='netflix', password='netflix123')


        users = [u1]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

Records exist uid netflix, or error.


In [174]:
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False

In [175]:
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    appName = input("Enter appName: ")
    personalUse = input("personalUse: ")
    favoriteFeature = input("favoriteFeature: ")
    password = input("password: ")
    uid = input("uid: ")

       
    # Initialize User object before date
    user = User(
        appName = appName,
        personalUse = personalUse,
        favoriteFeature = favoriteFeature,
        password = password,
        uid = uid
                )
    
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()


Created
 {'appName': '', 'personalUse': '', 'password': 'sha256$eQb...', 'favoriteFeature': '', 'uid': ''}


In [176]:
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [kush.read() for kush in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()


[{'appName': 'Netflix',
  'personalUse': 'Watching movies',
  'password': 'sha256$RI8...',
  'favoriteFeature': 'the recommendation feature',
  'uid': 'netflix'},
 {'appName': 'frwf',
  'personalUse': 'wf',
  'password': 'wf...',
  'favoriteFeature': 'wf',
  'uid': 'wfe'},
 {'appName': 'robinhood',
  'personalUse': 'trading stocks',
  'password': 'sha256$MHv...',
  'favoriteFeature': 'trading stocks',
  'uid': 'robinhood'},
 {'appName': 'clash of clans',
  'personalUse': 'playing videogames',
  'password': 'sha256$n1z...',
  'favoriteFeature': 'the color scheme',
  'uid': 'clash of clans'},
 {'appName': '',
  'personalUse': '',
  'password': 'sha256$eQb...',
  'favoriteFeature': '',
  'uid': ''}]

# delete functionality

In [177]:
def delete(): 
    uid = input("Enter uid of user that you want to delete: ")
    user = find_by_uid(uid)  
    with app.app_context():
        try:
            object = user.delete() 
            print(f"{uid} has been deleted. ")
            print(read())
        except:  
           (f"{uid} was not found.")
        
delete()

 has been deleted. 
[{'appName': 'Netflix', 'personalUse': 'Watching movies', 'password': 'sha256$RI8...', 'favoriteFeature': 'the recommendation feature', 'uid': 'netflix'}, {'appName': 'frwf', 'personalUse': 'wf', 'password': 'wf...', 'favoriteFeature': 'wf', 'uid': 'wfe'}, {'appName': 'robinhood', 'personalUse': 'trading stocks', 'password': 'sha256$MHv...', 'favoriteFeature': 'trading stocks', 'uid': 'robinhood'}, {'appName': 'clash of clans', 'personalUse': 'playing videogames', 'password': 'sha256$n1z...', 'favoriteFeature': 'the color scheme', 'uid': 'clash of clans'}]


# Update Functionality

In [178]:
def Updatey():

    uid = str(input("Type the uid of the user you want to change"))
    with app.app_context():
        user = User.query.filter_by(_uid =uid ).first()    
        
    
    with app.app_context():
        appName = str(input("New name?"))
        uid = str(input("New uid?"))
        password = str(input("New password?"))
        personalUse = str(input("new personal use?"))
        favoriteFeature = str(input("New Favorite feature?"))
        user._appName = appName
        user._uid = uid
        user._password = password
        user._personalUse = personalUse
        user._favoriteFeature = favoriteFeature
        x = user.update()
        db.session.commit()   
    
    print("User has been updated" )
        
Updatey()




User has been updated


In [179]:
print(read())

[{'appName': 'Netflix', 'personalUse': 'Watching movies', 'password': 'sha256$RI8...', 'favoriteFeature': 'the recommendation feature', 'uid': 'netflix'}, {'appName': 'google', 'personalUse': 'looking up things', 'password': 'google...', 'favoriteFeature': 'looking up things', 'uid': 'google'}, {'appName': 'robinhood', 'personalUse': 'trading stocks', 'password': 'sha256$MHv...', 'favoriteFeature': 'trading stocks', 'uid': 'robinhood'}, {'appName': 'clash of clans', 'personalUse': 'playing videogames', 'password': 'sha256$n1z...', 'favoriteFeature': 'the color scheme', 'uid': 'clash of clans'}]
